In [1]:
import pandas as pd 
from athena_to_pd import athena_query
import sqlite3
#By Book
from pathlib import Path
import os
import awswrangler as wr 
import boto3
from datetime import datetime

In [2]:
pwd = os.getcwd()

PROFILE = "vej-book"
DATABASE = "vtnlake_dev_curated_finance" 
#SQLITE_PATH = "../vtnlake_onedrive.db"
SQLITE_PATH = rf'D:\Work\SQLite\vtnlake_onedrive.db' # Book's Labtop
#SQLITE_PATH = rf'E:\Work\TradeSquare\SQlite Test\29102024\vtnlake_onedrive.db' # Book's PC

In [3]:
date_filter = athena_query(PROFILE, DATABASE,"""
    select 
        min(date_format(parse_datetime(_file_updated_date,'yyyy-MM-dd''T''HH:mm:ss''Z'),'%Y-%m-%d %H:%i:%s')) AS min_date,
        max(date_format(parse_datetime(_file_updated_date,'yyyy-MM-dd''T''HH:mm:ss''Z'),'%Y-%m-%d %H:%i:%s')) AS max_date
        
    from gl_header
""")

In [4]:
df_date_filter = pd.DataFrame(date_filter, columns = date_filter.columns)
min_date = df_date_filter.iloc[0, 0]
max_date = df_date_filter.iloc[0, 1]

print("Min Date Filter: ", min_date)
print("Max Date Filter: ", max_date)

Min Date Filter:  2024-11-04 13:29:23
Max Date Filter:  2024-11-05 01:45:11


In [6]:
'''
Select DISTINCT script for Athena (only PK)
'''

# Read the SQL template file (Book's PC)
with open(rf"{pwd}/sql_template/gl_header/select_dis_gl_header_athena.sql", "r") as file:
    query_template = file.read()

# Substitute placeholders with actual values
query_gl_header = query_template.format(min_date=min_date, max_date=max_date)

# Run the query with wr
select_gl_header_athena = wr.athena.read_sql_query(sql=query_gl_header, 
        database=DATABASE,
        boto3_session=boto3.session.Session(profile_name=PROFILE, region_name='ap-southeast-1')                      
                              )

##---------------------------------##
#global variables for sqlite
TABLE = "headers_full"

'''
Select DISTINCT script for SQLite (only PK)
'''

# Read the SQL template file
with open(rf"{pwd}/sql_template/gl_header/select_dis_gl_header_sqlite.sql", "r") as file:
    query_template = file.read()

# Substitute placeholders with actual values
SQLITE_QUERY = query_template.format(
    TABLE=TABLE,
    min_date=min_date,
    max_date=max_date
)    
# Execute the query
conn = sqlite3.connect(SQLITE_PATH)
df_select_sqlite = pd.read_sql_query(SQLITE_QUERY, conn).add_suffix('_sqlite')
conn.close()
select_gl_header_sqlite = df_select_sqlite


In [7]:
#sqlite
select_gl_header_sqlite.sort_values(by=['invoice_no_receipt_no_sqlite', 'payment_method_sqlite'], inplace=True)
select_gl_header_sqlite.reset_index(drop=True, inplace=True)
#athena
select_gl_header_athena.sort_values(by=['invoice_no_receipt_no', 'payment_method'], inplace=True)
select_gl_header_athena.reset_index(drop=True, inplace=True)

#Concat dataframe for Reconcilation
df_reconcile = pd.concat([select_gl_header_sqlite, select_gl_header_athena], axis=1)

#check status if suffix sqlite is match with athena using lambda for each column and row
df_reconcile['invoice_no_receipt_no_status'] = df_reconcile.apply(lambda x: 'Match' if x['invoice_no_receipt_no_sqlite'] == x['invoice_no_receipt_no'] else 'Mismatch', axis=1)
df_reconcile['payment_method_status'] = df_reconcile.apply(lambda x: 'Match' if x['payment_method_sqlite'] == x['payment_method'] else 'Mismatch', axis=1)
df_reconcile['status'] = df_reconcile.apply(lambda x: 'Match' if x['invoice_no_receipt_no_sqlite'] == x['invoice_no_receipt_no'] and x['payment_method_sqlite'] == x['payment_method'] else 'Mismatch', axis=1)

# df_reconcile[df_reconcile['status'] == 'Mismatch']

## Describe Statistics

In [8]:
stat_gl_header_sqlite =  pd.DataFrame(select_gl_header_sqlite.describe(include='all'))


# Convert all string values to hashed integers in a copy of the DataFrame
hashed_df = select_gl_header_sqlite.apply(lambda col: col.map(lambda x: hash(x) if isinstance(x, str) else x))

'''
SUM HASHED
'''
# Calculate the sum of each column and convert to a DataFrame row
sum_gl_header_sqlite = hashed_df.sum().to_frame().T
sum_gl_header_sqlite.index = ["sum_hashed"] 

'''
AVG HASHED
'''
avg_gl_header_sqlite = hashed_df.mean().to_frame().T
avg_gl_header_sqlite.index = ["avg_hashed"]

'''
MIN HASHED
'''
min_gl_header_sqlite = hashed_df.min().to_frame().T
min_gl_header_sqlite.index = ["min_hashed"]

'''
MAX HASHED
'''
max_gl_header_sqlite = hashed_df.max().to_frame().T
max_gl_header_sqlite.index = ["max_hashed"]

# Concatenate the statistics DataFrames
stat_select_gl_header_sqlite = pd.concat([stat_gl_header_sqlite, sum_gl_header_sqlite, avg_gl_header_sqlite, min_gl_header_sqlite, max_gl_header_sqlite], axis=0)


In [9]:
# Athena Statistics

stat_gl_header_athena =  pd.DataFrame(select_gl_header_athena.describe(include='all'))


# Convert all string values to hashed integers in a copy of the DataFrame
hashed_df = select_gl_header_athena.apply(lambda col: col.map(lambda x: hash(x) if isinstance(x, str) else x))

'''
SUM HASHED
'''
# Calculate the sum of each column and convert to a DataFrame row
sum_gl_header_athena = hashed_df.sum().to_frame().T
sum_gl_header_athena.index = ["sum_hashed"] 

'''
AVG HASHED
'''
avg_gl_header_athena = hashed_df.mean().to_frame().T
avg_gl_header_athena.index = ["avg_hashed"]

'''
MIN HASHED
'''
min_gl_header_athena = hashed_df.min().to_frame().T
min_gl_header_athena.index = ["min_hashed"]

'''
MAX HASHED
'''
max_gl_header_athena = hashed_df.max().to_frame().T
max_gl_header_athena.index = ["max_hashed"]

# Concatenate the statistics DataFrames
stat_select_gl_header_athena = pd.concat([stat_gl_header_athena, sum_gl_header_athena, avg_gl_header_athena, min_gl_header_athena, max_gl_header_athena], axis=0)



In [10]:
stat_select_gl_header_athena

,invoice_no_receipt_no,payment_method
count,525326,525326
unique,499211,11
top,D66016691,CRC
freq,4,172947
sum_hashed,-8217080412292853933,-7384661524365760264
avg_hashed,-8197402088582363.0,-1778721706085759488.0
min_hashed,-9223370346243070901,-9044453527849431452
max_hashed,9223318366835145502,7243502204763661199


In [11]:
# Compare statistics between SQLite and Athena
stat_reconclie = pd.concat([stat_select_gl_header_sqlite, stat_select_gl_header_athena], axis=1)

#check status if suffix sqlite is match with athena using lambda for each column and row
stat_reconclie['invoice_no_receipt_no_status'] = stat_reconclie.apply(lambda x: 'Match' if x['invoice_no_receipt_no_sqlite'] == x['invoice_no_receipt_no'] else 'Mismatch', axis=1)
stat_reconclie['payment_method_status'] = stat_reconclie.apply(lambda x: 'Match' if x['payment_method_sqlite'] == x['payment_method'] else 'Mismatch', axis=1)
stat_reconclie['status'] = stat_reconclie.apply(lambda x: 'Match' if x['invoice_no_receipt_no_sqlite'] == x['invoice_no_receipt_no'] and x['payment_method_sqlite'] == x['payment_method']  else 'Mismatch', axis=1)

In [12]:
### export to .csv file with sheets for each type of reconcilation
current_date = datetime.now().strftime('%Y%m%d')
output_path = rf"{pwd}/report/{current_date}_gl_header_reconcile.xlsx"
with pd.ExcelWriter(output_path) as writer:  
    df_reconcile.to_excel(writer, sheet_name='Data Reconcile', index=False)
    stat_reconclie.to_excel(writer, sheet_name='Stat Reconcile', index=True)